In [22]:
import pandas as pd
data = pd.read_pickle('testset.pickle')

In [23]:
# print first 5 data keys
print(len(list(data.keys())))

2470


In [25]:
list(list(data.keys())[2450:2455])

['https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml',
 'https://www.baseball-reference.com/boxes/SDN/SDN202210150.shtml',
 'https://www.baseball-reference.com/boxes/SEA/SEA202210150.shtml',
 'https://www.baseball-reference.com/boxes/CLE/CLE202210160.shtml',
 'https://www.baseball-reference.com/boxes/NYA/NYA202210180.shtml']

In [26]:
data['https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml']

{'game_page_url': 'https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml',
 'have_att': True,
 'start_time': datetime.datetime(2022, 10, 15, 14, 7),
 'day': 'Saturday',
 'month': 'October',
 'is_federal_holiday': 0,
 'duration': datetime.timedelta(seconds=11880),
 'venue': 'Citizens Bank Park',
 'at_night': False,
 'on_grass': True,
 'weather_description': ' 72° F, Wind 12mph out to Centerfield, Sunny, No Precipitation.',
 'temperature': 72,
 'wind_speed': 12,
 'weather': ' Sunny',
 'precipitation': 1,
 'umpires': ' HP - Stu Scheurwater, 1B - Chad Fairchild, 2B - Ryan Blakney, 3B - David Rackley, LF - Nic Lentz, RF - Bill Miller.',
 'attendance': 45660,
 'team1_name': 'AtlantaBraves',
 'team1_pre_win': 101,
 'team1_pre_loss': 60,
 'team1_pre_win_pct': 0.6273291925465838,
 'team1_players':                  Batting AB  R  H RBI BB SO PA    BA   OBP  ...     WPA  \
 0    Ronald Acuna Jr. RF  4  0  0   0  0  3  4  .333  .412  ...  -0.067   
 1      Dansby Swanson SS  4  0  0   0 